In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# FUNCTION FOR CALCULATING ANGLE
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
# SQUAT COUNTER
cap = cv2.VideoCapture(0)

# SQUAT counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        image2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image2.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image2.flags.writeable = True
        image2 = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
            
            # Calculate angle
            angle1 = calculate_angle(l_hip, l_knee, l_ankle)
            angle2 = calculate_angle(r_hip, r_knee, r_ankle)
            
            # Visualize angle
            cv2.putText(image, str(angle1), 
                           tuple(np.multiply(l_knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Squat counter logic
            if angle1 and angle2 > 170:
                stage = "Down"
            if angle1 < 150 and angle2 < 150 and stage =='Down' and abs(angle1-angle2) < 8:
                stage="Up"
                counter +=1
                #print(counter)
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
#         cv2.rectangle(image, (0,0), (225,73), (0,0,0), -1)
#         img = cv2.imread(image)
        x, y, w, h = 10, 10, 120, 80
        sub_img = image[y:y+h, x:x+w]
        white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 255
        res = cv2.addWeighted(sub_img, 0.5, white_rect, 0.6, 1.0)
        image[y:y+h, x:x+w] = res
        font = cv2.FONT_HERSHEY_TRIPLEX
        
        #a, b, c, d = 20, 10, 120, 80
        #sub_img2 = image2[b:b+d, a:a+c]
        #white_rect2 = np.ones(sub_img2.shape, dtype=np.uint8) * 255
        #res2 = cv2.addWeighted(sub_img2, 0.5, white_rect2, 0.6, 1.0)
       # image2[b:b+d, a:a+c] = res2
    
        
        # Stage data
        cv2.putText(sub_img, 'Direction', (12,18), 
                    font, 0.6, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(sub_img, stage, 
                    (12,60), 
                    font, 0.7, (0,0,0), 1, cv2.LINE_AA)
    
        x, y, w, h = 1150, 10, 120, 80
        sub_img = image[y:y+h, x:x+w]
        white_rect = np.ones(sub_img.shape, dtype=np.uint8) * 255
        res = cv2.addWeighted(sub_img, 0.5, white_rect, 0.6, 1.0)
        image[y:y+h, x:x+w] = res
        
        # Rep data
        cv2.putText(sub_img, 'REPS', (30,18), font, 0.6, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(sub_img, str(counter), 
                    (30,60), 
                    font, 0.8, (0,0,0), 1, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
